In [1]:
# Notes on reading the ugly light curve files:
# MHJD is modified heliocentric julian date  =  HJD - 2400000.5
# MAG is the mag on the 2MASS system
# MER is the mag err
# SRC is the src of photometry, 2 = Kaluzny Vband   3 = Kaluzny Bband
# -1 = Fourstar H    -2 = Fourstar Ks  -5 = Fourstar J
# NUM is the number of averaged measurements that went into MAG.  Should
# be unity for everything

%matplotlib inline
from collections import OrderedDict
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import prettyplotlib as ppl
jhk_dir = '../Omega_Cen_phot/'

In [2]:
df_orig = pd.read_csv('final_data_files/kaluzny_2004_rrl_reformat.csv')

In [3]:
ids = df_orig.id.astype(int).astype(str).str.zfill(3).values

In [4]:
def pad_nan(arr, length=12):
    npad = length - len(arr)
    if npad > 0:
        return np.concatenate([arr, [np.nan]*npad])
    else:
        return arr

In [5]:
lc_3p6 = OrderedDict()
for f in glob.glob('../calibrated_files/omegaCen_?_3p6um_rrl_matched_to_kaluzny.cut'):
    mjds = np.loadtxt(f.replace('_rrl_matched_to_kaluzny.cut','.mjds'))[:,1]
    df = pd.read_csv(f)
    for index, row, in df.iterrows():
        lc_3p6[row.ID.split('V')[-1] + '_mags'] = pad_nan(row.filter(regex='mag_').values)
        lc_3p6[row.ID.split('V')[-1] + '_merr'] = pad_nan(row.filter(regex='merr_').values)
        lc_3p6[row.ID.split('V')[-1] + '_mjds'] = sorted(mjds)
lc_4p5 = OrderedDict()
for f in glob.glob('../calibrated_files/omegaCen_?_4p5um_rrl_matched_to_kaluzny.cut'):
    mjds = np.loadtxt(f.replace('_rrl_matched_to_kaluzny.cut','.mjds'))[:,1]
    df = pd.read_csv(f)
    for index, row, in df.iterrows():
        lc_4p5[row.ID.split('V')[-1] + '_mags'] = pad_nan(row.filter(regex='mag_').values)
        lc_4p5[row.ID.split('V')[-1] + '_merr'] = pad_nan(row.filter(regex='merr_').values)
        lc_4p5[row.ID.split('V')[-1] + '_mjds'] = sorted(mjds)

In [23]:
def phase(mjd_dict,per):
    datemin = np.min(np.concatenate(mjd_dict.values()))
    for k,v in mjd_dict.iteritems():
        v -= datemin
        v /= per
        v -= v.astype(int)
        mjd_dict[k] = v
    return mjd_dict

In [74]:
lc_dict = OrderedDict()
offset_dict = {'j':4,'h':3,'k':2,'3':1,'4':0}
name_dict = {'j':'J','h':'H','k':'Ks','3':'[3.6]','4':'[4.5]'}
cmap = plt.cm.get_cmap('gist_rainbow')
plot_colors = list(reversed([cmap(i) for i in np.linspace(0,0.91,5)]))
color_dict = {'j':plot_colors[0],'h':plot_colors[1],'k':plot_colors[2],'3':plot_colors[3],'4':plot_colors[4]}
for obj in ids:
    fname = os.path.join(jhk_dir, '{}.txt'.format(obj))
    per = df_orig.per[df_orig.id == float(obj)].values[0]
    mjd_dict = OrderedDict()
    mags_dict = OrderedDict()
    merr_dict = OrderedDict()
    n_data = 0
    mjds_name = '{}_mjds'.format(int(obj))
    mags_name = '{}_mags'.format(int(obj))
    merr_name = '{}_merr'.format(int(obj))
    if os.path.exists(fname):
        df = pd.read_csv(fname, delim_whitespace=True, names=['mjd','mag','merr','band','n'])
        df_j = df[df.band==-5]
        df_h = df[df.band==-1]
        df_k = df[df.band==-2]
        if df_j.shape[0] > 0:
            mjd_dict['j'] = df_j.mjd.values
            mags_dict['j'] = df_j.mag.values
            merr_dict['j'] = df_j.merr.values
            n_data += 1
        if df_h.shape[0] > 0:
            mjd_dict['h'] = df_h.mjd.values
            mags_dict['h'] = df_h.mag.values
            merr_dict['h'] = df_h.merr.values
            n_data += 1
        if df_j.shape[0] > 0:
            mjd_dict['k'] = df_k.mjd.values
            mags_dict['k'] = df_k.mag.values
            merr_dict['k'] = df_k.merr.values
            n_data += 1
    else:
        print 'file does not exist:', obj
    if mjds_name in lc_3p6:
        mjd_dict['3'] = lc_3p6[mjds_name]
        mags_dict['3'] = lc_3p6[mags_name]
        merr_dict['3'] = lc_3p6[merr_name]
        n_data += 1
    if mjds_name in lc_4p5:
        mjd_dict['4'] = lc_4p5[mjds_name]
        mags_dict['4'] = lc_4p5[mags_name]
        merr_dict['4'] = lc_4p5[merr_name]
        n_data += 1
    if n_data > 0:
        phased_dict = phase(mjd_dict,per)
        for k,v in phased_dict.iteritems():
            basename = '{}_{}'.format(int(obj),k)
            lc_dict[basename + '_phase'] = pad_nan(v)
            lc_dict[basename + '_mags'] = pad_nan(mags_dict[k])
#         fig = plt.figure(figsize=(6,6))
#         ax = fig.add_axes([0.1,0.09,0.725,0.85])
#         for k,v in phased_dict.iteritems():
#             p0 = ppl.errorbar(ax,v,mags_dict[k]-offset_dict[k],yerr=merr_dict[k],color=color_dict[k],
#                          fmt='o', label='{}-{}'.format(name_dict[k],offset_dict[k]))
#             ppl.errorbar(ax,v+1,mags_dict[k]-offset_dict[k],yerr=merr_dict[k],color=color_dict[k],
#                          fmt='o', label=None)
#             ppl.errorbar(ax,v+2,mags_dict[k]-offset_dict[k],yerr=merr_dict[k],color=color_dict[k],
#                          fmt='o', label=None)
#         maglist = np.concatenate([mags_dict[k] - offset_dict[k] for k in mags_dict.keys()])
#         maglist = maglist.astype(float)[np.isfinite(maglist.astype(float))]
#         ax.set_ylim(np.nanmax(maglist)+0.1,np.nanmin(maglist)-0.1)
#         ax.set_xlim(-0.1,3.1)
#         ax.set_xlabel('Phase')
#         ax.set_ylabel('Mag')
#         ax.set_title('V{} light curves'.format(int(obj)))
#         try:
#             leg = ppl.legend(loc='upper left',bbox_to_anchor=(1, 1),numpoints=1,handlelength=0)
#         except:
#             pass
#         fig.savefig('../light_curves/all_{}_lightcurves.pdf'.format(int(obj)))
    else:
        print 'no data for', obj


no data for 055
no data for 069
file does not exist: 134
no data for 134
file does not exist: 159
no data for 159
file does not exist: 160
no data for 160
file does not exist: 181
no data for 181
file does not exist: 182
no data for 182
file does not exist: 183
no data for 183
file does not exist: 281
no data for 281
file does not exist: 283
no data for 283
file does not exist: 351
no data for 351


In [75]:
lc_df = pd.DataFrame(lc_dict)
lc_df.to_csv('final_data_files/lightcurves.csv',index=False)